In [1]:
#!pip install google-cloud-storage
import pandas as pd
from google.cloud import storage
# import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hedwig/00_Data/msca310019-capstone-49b3-6876cdba6fac.json"

In [14]:
#!pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
client = storage.Client()
bucket = client.get_bucket('user-scripts-msca310019-capstone-49b3')

In [3]:
blob = bucket.blob('data/conversation_emails.csv')
content = blob.download_as_string()

In [4]:
import io
df = pd.read_csv(io.BytesIO(content))

In [6]:
df.drop_duplicates(subset=['message'], keep='first',inplace=True)

In [16]:
thread_counts=df.groupby(['thread'], as_index=False)['message'].agg('count')
thread_counts.head(20)

,thread,message
0,0,8432
1,1,2546
2,2,68
3,3,137
4,4,1
5,5,2
6,6,2
7,7,3
8,8,3
9,9,4


In [17]:
thread_multi=thread_counts[thread_counts.message>=2]
thread_multi.drop([0,1], inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
multi_threads_list=thread_multi.thread.to_list()

In [ ]:
df['flg_multi']=df['thread'].apply(lambda x: 1 if x in multi_threads_list else 0)

In [ ]:
df_message_v3=df_message_v2[df_message_v2.flg_multi==1]
df_message_v3.head()